In [29]:
import numpy as np

from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [30]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
QiskitRuntimeService.save_account(channel="ibm_quantum",token="5bdd6c2a73e84f421422bf22890663ce0c83b96215f059ce6d1af3d00218c1611418dc4cf86a33554518d39a5749435c45ecbe558d19ac2c3013ec978207840c",overwrite=True)
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
backend.name

'ibm_kyoto'

In [31]:
#general roadmap for VQE

#choose Hamiltonian
Hamiltonian = SparsePauliOp.from_list([("XZ", -1.0), ("ZX", -1.0), ("YY", 1.0), ("II", 1.0)])

#choose ansatz

ansatz = EfficientSU2(Hamiltonian.num_qubits, reps=1)

#format with pass manager

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

pm = generate_preset_pass_manager(target=backend.target,optimization_level=3)
ISA_ansatz = pm.run(ansatz)
ISA_Hamiltonian = Hamiltonian.apply_layout(layout=ISA_ansatz.layout)


def cost_function(params,ansatz,hamiltonian,estimator):
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    cost = result[0].data.evs[0]

    return cost



In [32]:
session = Session(backend=backend)
estimator = Estimator(session=session)
estimator.options.default_shots = 1000

sampler = Sampler(session=session)
sampler.options.default_shots = 100


x0 = 2*np.pi*np.random.rand(ISA_ansatz.num_parameters)



In [33]:
res = minimize(cost_function, x0, args=(ISA_ansatz, ISA_Hamiltonian, estimator), method="COBYLA")

/Users/jonahsachs/opt/miniconda3/envs/myQenv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 623.1034379365619 quantum seconds, but you only have 519 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)
/Users/jonahsachs/opt/miniconda3/envs/myQenv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 623.1034379365619 quantum seconds, but you only have 514 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)
/Users/jonahsachs/opt/miniconda3/envs/myQenv/lib/python3.12/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 623.1034379365619 quantum seconds, but you only have 509 quantum sec

: 